In [1]:
import numpy as np
import pyhf
import timeit

In [2]:
def generate_source_poisson(n_bins):
    """
    Create the source structure for the given number of bins.
    Sample from a Poisson distribution
    Args:
        n_bins: `list` of number of bins
    Returns:
        source
    """
    np.random.seed(0)  # Fix seed for reproducibility
    binning = [n_bins, -0.5, n_bins + 0.5]
    data = np.random.poisson(120.0, n_bins).tolist()
    bkg = np.random.poisson(100.0, n_bins).tolist()
    bkgerr = np.random.poisson(10.0, n_bins).tolist()
    sig = np.random.poisson(30.0, n_bins).tolist()

    source = {
        'binning': binning,
        'bindata': {'data': data, 'bkg': bkg, 'bkgerr': bkgerr, 'sig': sig},
    }
    return source

def timeit_setup(n_bins, backend_name, mode):
    source = generate_source_poisson(n_bins)
    model = pyhf.simplemodels.hepdata_like(
        source['bindata']['sig'], source['bindata']['bkg'], source['bindata']['bkgerr']
    )
    pyhf.set_backend(backend_name)
    data = pyhf.tensorlib.astensor(source['bindata']['data'] + model.config.auxdata)
    test_poi = 1.0
    return test_poi, data, model
    
def timeit_run(test_mu, data, model):
    # tensorlib is set in timeit_setup
    return pyhf.infer.hypotest(
        test_mu,
        data,
        model
    )

def generate_setup_string(n_bins, backend_name, mode):
    setup_string = "from __main__ import generate_source_poisson, timeit_setup, timeit_run; "
    setup_string += "import numpy as np; "
    if mode.lower() == "cpu":
        if backend_name.lower() == "tensorflow":
            # Hide the GPU from TensorFlow
            setup_string += "import tensorflow as tf; "
            setup_string += "tf.config.set_visible_devices([], 'GPU'); "
    elif mode.lower() != "gpu":
        return "ERROR"
    setup_string += f"test_mu, data, model = timeit_setup({n_bins}, '{backend_name}', '{mode}')"
    return setup_string

In [3]:
n_bins = 10
setup_string = generate_setup_string(n_bins, "numpy", "CPU")
# setup_string = generate_setup_string(n_bins, "tensorflow", "CPU")
n_runs = 10
exec_time = timeit.timeit('timeit_run(test_mu, data, model)', number=n_runs, setup=setup_string)
exec_time, exec_time/n_runs

(1.6086194328963757, 0.16086194328963757)

In [4]:
results = [("n_bins", "total_time", "n_runs", "time_run")]
# np.linspace(500,7000,14)
bin_range = np.linspace(1,10,5)
print(bin_range)

backend_name = "numpy"
mode = "CPU"
n_runs = 10

for n_bins in bin_range:
    n_bins = int(n_bins)
    setup_string = generate_setup_string(n_bins, backend_name, mode)
    exec_time = timeit.timeit('timeit_run(test_mu, data, model)', number=n_runs, setup=setup_string)
    
    results.append((n_bins, exec_time, n_runs, exec_time/n_runs))
    print('N: {} time: {} time per run: {}'.format(n_bins, exec_time, exec_time/n_runs))

[ 1.    3.25  5.5   7.75 10.  ]
N: 1 time: 0.16665149992331862 time per run: 0.016665149992331863
N: 3 time: 0.4205212080851197 time per run: 0.04205212080851197
N: 5 time: 0.7023275690153241 time per run: 0.07023275690153241
N: 7 time: 1.04108702018857 time per run: 0.104108702018857
N: 10 time: 1.590267052873969 time per run: 0.1590267052873969


In [5]:
setup_string

"from __main__ import generate_source_poisson, timeit_setup, timeit_run; import numpy as np; test_mu, data, model = timeit_setup(10, 'numpy', 'CPU')"

In [6]:
results

[('n_bins', 'total_time', 'n_runs', 'time_run'),
 (1, 0.16665149992331862, 10, 0.016665149992331863),
 (3, 0.4205212080851197, 10, 0.04205212080851197),
 (5, 0.7023275690153241, 10, 0.07023275690153241),
 (7, 1.04108702018857, 10, 0.104108702018857),
 (10, 1.590267052873969, 10, 0.1590267052873969)]